# Accessing NCBI database for data retrieval

NCBI stands for National Center for Biotechnology Information.
They possess many databases for biological data, as well as journals
and articles of a wide variety of topics. It is common for sequencing
data to be uploaded to the website, as well as other data from 
biological experiments.

Biopython has a built-in object to access NCBI and retrieve data from 
their databases. That is the `Entrez` object, which we'll explore below:

In [1]:
from Bio import Entrez, SeqIO
Entrez.email = "guilhormo.47@gmail.com" # setting an email is good practice

## `EInfo`: obtaining information about Entrez databases:

EInfo provides interesting information about each of Entrez' databases,
such as field index term counts, last update, available links, and 
databases' names. We'll obtain here all databases' names:

In [34]:
stream = Entrez.einfo()
result = stream.read()
stream.close()

The `result` variable now contains a list of databases in XML format (
as is standard for Entrez search return types):

In [35]:
result

b'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20190110//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20190110/einfo.dtd">\n<eInfoResult>\n<DbList>\n\n\t<DbName>pubmed</DbName>\n\t<DbName>protein</DbName>\n\t<DbName>nuccore</DbName>\n\t<DbName>ipg</DbName>\n\t<DbName>nucleotide</DbName>\n\t<DbName>structure</DbName>\n\t<DbName>genome</DbName>\n\t<DbName>annotinfo</DbName>\n\t<DbName>assembly</DbName>\n\t<DbName>bioproject</DbName>\n\t<DbName>biosample</DbName>\n\t<DbName>blastdbinfo</DbName>\n\t<DbName>books</DbName>\n\t<DbName>cdd</DbName>\n\t<DbName>clinvar</DbName>\n\t<DbName>gap</DbName>\n\t<DbName>gapplus</DbName>\n\t<DbName>grasp</DbName>\n\t<DbName>dbvar</DbName>\n\t<DbName>gene</DbName>\n\t<DbName>gds</DbName>\n\t<DbName>geoprofiles</DbName>\n\t<DbName>medgen</DbName>\n\t<DbName>mesh</DbName>\n\t<DbName>nlmcatalog</DbName>\n\t<DbName>omim</DbName>\n\t<DbName>orgtrack</DbName>\n\t<DbName>pmc</DbName>\n\t<DbName>proteinclusters</DbNa

We can parse this result with the `Entrez` object, returning a python object:


In [36]:
stream = Entrez.einfo()
record = Entrez.read(stream)

This returns a dictionary with one key, `'DbList'`. This key value is the
list of all available databases:

In [37]:
record['DbList']

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'medgen', 'mesh', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']

For each database, we can use `einfo()` again to gain more information:

In [2]:
stream = Entrez.einfo(db='nucleotide')
record = Entrez.read(stream)
record['DbInfo'].keys()

dict_keys(['DbName', 'MenuName', 'Description', 'DbBuild', 'Count', 'LastUpdate', 'FieldList', 'LinkList'])

In [39]:
record['DbInfo']['Count']

'688445866'

One really useful data from this object return from `einfo(db)` is the 'FieldList',
which gives us a list of possible search fields to use with ESearch:

In [4]:
for field in record['DbInfo']['FieldList']:
  print(f"{field['Name']}, {field['FullName']}, {field['Description']}")

ALL, All Fields, All terms from all searchable fields
UID, UID, Unique number assigned to each sequence
FILT, Filter, Limits the records
WORD, Text Word, Free text associated with record
TITL, Title, Words in definition line
KYWD, Keyword, Nonstandardized terms provided by submitter
AUTH, Author, Author(s) of publication
JOUR, Journal, Journal abbreviation of publication
VOL, Volume, Volume number of publication
ISS, Issue, Issue number of publication
PAGE, Page Number, Page number(s) of publication
ORGN, Organism, Scientific and common names of organism, and all higher levels of taxonomy
ACCN, Accession, Accession number of sequence
PACC, Primary Accession, Does not include retired secondary accessions
GENE, Gene Name, Name of gene associated with sequence
PROT, Protein Name, Name of protein associated with sequence
ECNO, EC/RN Number, EC number for enzyme or CAS registry number
PDAT, Publication Date, Date sequence added to GenBank
MDAT, Modification Date, Date of last update
SUBS, S

This is used to restrict our search within a specific field, such as with
`Jones[AUTH]` to limit our search to the author field, or `Sanger[AFFL]` to
limit out search to author's affiliated to the Sanger Centre.

In [2]:
handle = Entrez.esearch(db="nucleotide", term='CRT[Gene Name] AND "Plasmodium falciparum"[Organism]')
rec_list = Entrez.read(handle)
if int(rec_list['RetMax']) < int(rec_list['Count']):
    handle = Entrez.esearch(db="nucleotide", term='CRT[Gene Name] AND "Plasmodium falciparum"[Organism]', retmax=rec_list['Count'])
    rec_list = Entrez.read(handle)

In [3]:
id_list = rec_list['IdList']
hdl = Entrez.efetch(db='nucleotide', id=id_list, rettype='gb', retmax=rec_list['Count'])

In [4]:
recs = list(SeqIO.parse(hdl, 'gb'))

In [5]:
for rec in recs:
    if rec.name == 'KM288867':
        break
print(rec.name)
print(rec.description)

KM288867
Plasmodium falciparum clone PF3D7_0709000 chloroquine resistance transporter (CRT) gene, complete cds


In [6]:
for feature in rec.features:
    if feature.type == 'gene':
        print(feature.qualifiers['gene'])
    elif feature.type == 'exon':
        loc = feature.location
        print('Exon', loc.start, loc.end, loc.strand)
    else:
        print('not processed:\n%s' % feature)

not processed:
type: source
location: [0:10000](+)
qualifiers:
    Key: clone, Value: ['PF3D7_0709000']
    Key: db_xref, Value: ['taxon:5833']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Plasmodium falciparum']

['CRT']
not processed:
type: mRNA
location: join{[2751:3543](+), [3720:3989](+), [4168:4341](+), [4513:4646](+), [4799:4871](+), [4994:5070](+), [5166:5249](+), [5376:5427](+), [5564:5621](+), [5769:5862](+), [6055:6100](+), [6247:6302](+), [6471:7598](+)}
qualifiers:
    Key: gene, Value: ['CRT']
    Key: product, Value: ['chloroquine resistance transporter']

not processed:
type: 5'UTR
location: [2751:3452](+)
qualifiers:
    Key: gene, Value: ['CRT']

not processed:
type: primer_bind
location: [2935:2958](+)
qualifiers:

not processed:
type: primer_bind
location: [3094:3121](+)
qualifiers:

not processed:
type: CDS
location: join{[3452:3543](+), [3720:3989](+), [4168:4341](+), [4513:4646](+), [4799:4871](+), [4994:5070](+), [5166:5249](+), [5376:54

In [7]:
for name, value in rec.annotations.items():
  print(f'{name}={value}')

molecule_type=DNA
topology=linear
data_file_division=INV
date=12-NOV-2014
accessions=['KM288867']
sequence_version=1
keywords=['']
source=Plasmodium falciparum (malaria parasite P. falciparum)
organism=Plasmodium falciparum
taxonomy=['Eukaryota', 'Sar', 'Alveolata', 'Apicomplexa', 'Aconoidasida', 'Haemosporida', 'Plasmodiidae', 'Plasmodium', 'Plasmodium (Laverania)']
references=[Reference(title='Versatile control of Plasmodium falciparum gene expression with an inducible protein-RNA interaction', ...), Reference(title='Direct Submission', ...)]


In [8]:
print(rec.seq)

ATATGTAAAACCAAAATAAATTAAACAGAATTTATTTTTAAAAGATTTATTTGTAACAATATTACCATGATGATTTATTAAAGTAAAATCACCACCTATTAATGGTTTTCCTATACTTTCCATTGTAGTTTTTCCAAAACCTTTTTTTTTATTTTTCTCATTTTGTAATAAATATAAATATAAATATAATGCTGGTATCGTTAATGATAAATTAATACCTAAACATTTCCATGTTAAAAATATATTTTTCTTTTTCGACTTTTTTTTATTATCATTTTCATCCTCAACTTTTTTACTCATACTACTATAATCATTTCTTATAAAACTTTTATAAAATATATTTTCCCTTTTTTTATAATTTCTACTAGTATCATAAACAATATTTCTTATATTCAATAATACATTAAAATTGTTATATCCATTTTTTTTTTTATTACATATTTTGTTTAAGCTATTTAAAAGCACCTTATTCATTTTTGAAATAATGAAACAGCAAAAAAACAAAAATAACGTAAACAATTATGATGTACATAAAATATATATATATATATATATATATATATATTATTTATATATTATTCACATTTACATTTATTTATTTATATCTTCCTATTTTATACGATAAAATAAATTCGATTTAAATAAAATATACGTTAAATTAAATTCTTATATCTTAACATTTCGACATACATTTATTAATTGATTTTATTTTATTTATTTTGTTATATCCATATAAATAAACGTATCGCACATGAGTTTTTTTTTTTTTTTTTTTTTTTTTTTATATAATACTTTTTAATAGTGAAACACATACATATTTTTTACCATTTTAAATATATATATGGTGTACCATTTTGGGTTAATATATTTTTTGTAATTATATATTAATATATACATAATAAATATAAATTAATATTATATATATATATATATATATATATATAAATAATTGGAGTGTCCTATTTTGAAGCTATGTCCTTTGGAATACAGCACTTTACAA

## Performing basic sequence analysis

We will be using the human **lactase** gene as an example. We will use the
same methods as above for retrieving it.

In [5]:
from Bio import Entrez, SeqIO, SeqRecord
Entrez.email="guilhormo.47@gmail.com"
hdl = Entrez.efetch(db='nucleotide', id=['NM_002299'], rettype='gb') # lactase gene
gb_rec = SeqIO.read(hdl, 'gb')

Now that we have the record, we'll extract the gene sequence, even though it
contains more than that.

In [6]:
for feature in gb_rec.features:
  if feature.type == 'CDS':
    location = feature.location # Note translation existing
cds = SeqRecord.SeqRecord(gb_rec.seq[location.start:location.end], 'NM_002299', description='LCT CDS only')

Now our sequence is available in the Biopython sequence record.

We'll now proceed by saving the sequence to a FASTA file:

In [ ]:
from Bio import SeqIO
with open('example.fasta', 'w') as w_hdl:
  SeqIO.write([cds], w_hdl, 'fasta')

The Seq.IO write method takes a list of sequences to write. In our case it is just one,
but we should be careful - if we plan to write lots of sequences, an iterator
will allocate less memory from our computer.

In most situations we will have the sequence loaded on the disk and will
want to read it. For that, we have the `SeqIO.read()` method:

In [ ]:
recs = SeqIO.parse('example.fasta', 'fasta')
for rec in recs:
  seq = rec.seq
  print(rec.description)
  print(seq[:10])

As we now have pure DNA, we can transcribe it:

In [ ]:
rna = seq.transcribe()
print(rna)

And then, we can translate the gene into a protein:

In [ ]:
prot = seq.translate()
print(prot)